In [1]:
from sqlalchemy import create_engine
from getpass import getpass
import numpy as np 
import sklearn as skl
import math
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import r2_score
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestClassifier

secret = getpass('Enter the secret value: ')

Enter the secret value: ········


In [2]:
#pip install psycopg2

In [3]:
import psycopg2
connection = psycopg2.connect(
    host = 'housingdatabase.chkn2wwlsbys.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = secret,
    database='NYChouses'
    )
cursor=connection.cursor()


In [4]:
import pandas as pd

pd.set_option('display.max_colwidth',1000)

In [5]:
sql = """
SELECT *
FROM fulloutputhousing
"""
fulloutpuhousing = pd.read_sql(sql, con=connection)

In [6]:
#table for M/L model based of test data share with Toral
house2data = fulloutpuhousing[['house_price', 'address', 'zipcode', 'date', 'beds', 'bath','sq_ft', 'boro', 'employed', 'unemployed', 'income_over_100k', 'income_under_100k']]
house2data.columns = ['price','address','zip','date','beds','bath','sq_ft','boro','Employed','Unemployed','income above 100k','income below 100k']
house2data


,price,address,zip,date,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k
0,244000,"4205 48th Ave APT 3G, Flushing, NY 11377",11377,2020-10-16,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040
1,680000,"14404 76th Rd, Flushing, NY 11367",11367,2020-10-16,3,3.0,1980,Queens County,1099996,39401,166650,1015040
2,900000,"35 88th St, Brooklyn, NY 11209",11209,2020-10-16,3,1.0,1520,Kings County,1149461,48699,188640,1067440
3,625000,"11128 130th St, Jamaica, NY 11420",11420,2020-10-16,3,2.0,1440,Queens County,1099996,39401,166650,1015040
4,1720000,"20102 32nd Ave, Flushing, NY 11361",11361,2020-10-16,2,1.0,1406,Queens County,1099996,39401,166650,1015040
...,...,...,...,...,...,...,...,...,...,...,...,...
72657,317554,"2835 Webb Ave, Bronx, NY 10468",10468,2020-09-11,2,2.0,83000,Bronx County,565642,32035,55180,173430
72658,157000,"2640 Marion Ave APT 6H, Bronx, NY 10458",10458,2020-08-18,2,1.0,91075,Bronx County,565642,32035,55180,173430
72659,11500,"1711 Davidson Ave, Bronx, NY 10453",10453,2020-08-07,0,0.0,98700,Bronx County,565642,32035,55180,173430
72660,180887,"871 Elton Ave APT 2, Bronx, NY 10451",10451,2020-08-24,0,0.0,99240,Bronx County,565642,32035,55180,173430


In [7]:
house2data.groupby('boro').mean()

,price,zip,bath,sq_ft,Employed,Unemployed,income above 100k,income below 100k
boro,,,,,,,,
Bronx County,5.021488e+05,10464.877898,1.732945,3375.714065,565642.0,32035.0,55180.0,173430.0
Kings County,1.415096e+06,11228.788652,1.635106,3836.717730,1149461.0,48699.0,188640.0,1067440.0
New York County,2.200729e+06,10019.301713,1.467185,6219.493390,879665.0,31973.0,300250.0,594060.0
Queens County,7.026108e+05,11389.757455,1.862737,4115.249912,1099996.0,39401.0,166650.0,1015040.0
Richmond County,5.824694e+05,10308.421703,2.323308,1830.938187,213734.0,8585.0,55180.0,173430.0


In [8]:
df = house2data
df.head()

,price,address,zip,date,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k
0,244000,"4205 48th Ave APT 3G, Flushing, NY 11377",11377,2020-10-16,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040
1,680000,"14404 76th Rd, Flushing, NY 11367",11367,2020-10-16,3,3.0,1980,Queens County,1099996,39401,166650,1015040
2,900000,"35 88th St, Brooklyn, NY 11209",11209,2020-10-16,3,1.0,1520,Kings County,1149461,48699,188640,1067440
3,625000,"11128 130th St, Jamaica, NY 11420",11420,2020-10-16,3,2.0,1440,Queens County,1099996,39401,166650,1015040
4,1720000,"20102 32nd Ave, Flushing, NY 11361",11361,2020-10-16,2,1.0,1406,Queens County,1099996,39401,166650,1015040


In [9]:
# Print out the boro value counts
boro_counts = df.boro.value_counts()
boro_counts

Queens County                                         45608
New York County                                       21484
Kings County                                           2820
Richmond County                                        1456
Bronx County                                           1294
Name: boro, dtype: int64

In [10]:
new_df = df.drop(['address', 'date'], axis=1)
new_df.head()


,price,zip,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k
0,244000,11377,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040
1,680000,11367,3,3.0,1980,Queens County,1099996,39401,166650,1015040
2,900000,11209,3,1.0,1520,Kings County,1149461,48699,188640,1067440
3,625000,11420,3,2.0,1440,Queens County,1099996,39401,166650,1015040
4,1720000,11361,2,1.0,1406,Queens County,1099996,39401,166650,1015040


In [11]:
new_df['total_emp'] = new_df['Employed'] + new_df['Unemployed']
new_df.head()

,price,zip,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k,total_emp
0,244000,11377,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040,1139397
1,680000,11367,3,3.0,1980,Queens County,1099996,39401,166650,1015040,1139397
2,900000,11209,3,1.0,1520,Kings County,1149461,48699,188640,1067440,1198160
3,625000,11420,3,2.0,1440,Queens County,1099996,39401,166650,1015040,1139397
4,1720000,11361,2,1.0,1406,Queens County,1099996,39401,166650,1015040,1139397


In [12]:
new_df['Unemployment_rate'] = new_df['Unemployed']/new_df['total_emp']
new_df.head()

,price,zip,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k,total_emp,Unemployment_rate
0,244000,11377,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040,1139397,0.034581
1,680000,11367,3,3.0,1980,Queens County,1099996,39401,166650,1015040,1139397,0.034581
2,900000,11209,3,1.0,1520,Kings County,1149461,48699,188640,1067440,1198160,0.040645
3,625000,11420,3,2.0,1440,Queens County,1099996,39401,166650,1015040,1139397,0.034581
4,1720000,11361,2,1.0,1406,Queens County,1099996,39401,166650,1015040,1139397,0.034581


In [13]:
newrate_df = new_df.drop(['Employed', 'Unemployed', 'total_emp'], axis=1)
newrate_df.head()

,price,zip,beds,bath,sq_ft,boro,income above 100k,income below 100k,Unemployment_rate
0,244000,11377,Studio,1.0,53400,Queens County,166650,1015040,0.034581
1,680000,11367,3,3.0,1980,Queens County,166650,1015040,0.034581
2,900000,11209,3,1.0,1520,Kings County,188640,1067440,0.040645
3,625000,11420,3,2.0,1440,Queens County,166650,1015040,0.034581
4,1720000,11361,2,1.0,1406,Queens County,166650,1015040,0.034581


In [14]:
newrate_df['income2'] = newrate_df['income above 100k'] + newrate_df['income below 100k']
newrate_df.head()

,price,zip,beds,bath,sq_ft,boro,income above 100k,income below 100k,Unemployment_rate,income2
0,244000,11377,Studio,1.0,53400,Queens County,166650,1015040,0.034581,1181690
1,680000,11367,3,3.0,1980,Queens County,166650,1015040,0.034581,1181690
2,900000,11209,3,1.0,1520,Kings County,188640,1067440,0.040645,1256080
3,625000,11420,3,2.0,1440,Queens County,166650,1015040,0.034581,1181690
4,1720000,11361,2,1.0,1406,Queens County,166650,1015040,0.034581,1181690


In [15]:
newrate_df['income_rate'] = newrate_df['income above 100k']/newrate_df['income2']
newrate_df.head()

,price,zip,beds,bath,sq_ft,boro,income above 100k,income below 100k,Unemployment_rate,income2,income_rate
0,244000,11377,Studio,1.0,53400,Queens County,166650,1015040,0.034581,1181690,0.141027
1,680000,11367,3,3.0,1980,Queens County,166650,1015040,0.034581,1181690,0.141027
2,900000,11209,3,1.0,1520,Kings County,188640,1067440,0.040645,1256080,0.150182
3,625000,11420,3,2.0,1440,Queens County,166650,1015040,0.034581,1181690,0.141027
4,1720000,11361,2,1.0,1406,Queens County,166650,1015040,0.034581,1181690,0.141027


In [16]:
newrates_df = newrate_df.drop(['income above 100k', 'income below 100k', 'income2'], axis=1)
newrates_df.head()

,price,zip,beds,bath,sq_ft,boro,Unemployment_rate,income_rate
0,244000,11377,Studio,1.0,53400,Queens County,0.034581,0.141027
1,680000,11367,3,3.0,1980,Queens County,0.034581,0.141027
2,900000,11209,3,1.0,1520,Kings County,0.040645,0.150182
3,625000,11420,3,2.0,1440,Queens County,0.034581,0.141027
4,1720000,11361,2,1.0,1406,Queens County,0.034581,0.141027


In [17]:
list(newrates_df.iloc[2])

[900000,
 11209,
 '3',
 1.0,
 1520,
 'Kings County                                      ',
 0.040644822060492754,
 0.15018151710082162]

In [18]:
newrates_df.loc[newrate_df.beds == "Studio", "beds"] = "0"
newrates_df.head()

,price,zip,beds,bath,sq_ft,boro,Unemployment_rate,income_rate
0,244000,11377,0,1.0,53400,Queens County,0.034581,0.141027
1,680000,11367,3,3.0,1980,Queens County,0.034581,0.141027
2,900000,11209,3,1.0,1520,Kings County,0.040645,0.150182
3,625000,11420,3,2.0,1440,Queens County,0.034581,0.141027
4,1720000,11361,2,1.0,1406,Queens County,0.034581,0.141027


In [19]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False,drop ="first")

In [20]:
# Fit the encoder and produce encoded DataFrame
boro_df = pd.DataFrame(enc.fit_transform(newrates_df.boro.values.reshape(-1,1)))

In [21]:
newrates_df.boro.values.reshape(-1,1)

array([['Queens County                                     '],
       ['Queens County                                     '],
       ['Kings County                                      '],
       ...,
       ['Bronx County                                      '],
       ['Bronx County                                      '],
       ['Bronx County                                      ']],
      dtype=object)

In [22]:
# Rename encoded columns
boro_df.columns = enc.get_feature_names(['boro'])
boro_df.head()

,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [23]:
# Merge the two DataFrames together and drop the boro column
new2_df=newrates_df.merge(boro_df,left_index=True,right_index=True).drop("boro",1)
new2_df.head()

,price,zip,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County
0,244000,11377,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0
1,680000,11367,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0
2,900000,11209,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0
3,625000,11420,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0
4,1720000,11361,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0


In [24]:

#Find null Values

for column in new2_df.columns:
    print(f"Column {column} has {new2_df[column].isnull().sum()} null values")

Column price has 0 null values
Column zip has 0 null values
Column beds has 0 null values
Column bath has 218 null values
Column sq_ft has 0 null values
Column Unemployment_rate has 0 null values
Column income_rate has 0 null values
Column boro_Kings County                                       has 0 null values
Column boro_New York County                                    has 0 null values
Column boro_Queens County                                      has 0 null values
Column boro_Richmond County                                    has 0 null values


In [25]:
# Drop null rows
new2_df = new2_df.dropna()


In [26]:
#Find null Values

for column in new2_df.columns:
    print(f"Column {column} has {new2_df[column].isnull().sum()} null values")

Column price has 0 null values
Column zip has 0 null values
Column beds has 0 null values
Column bath has 0 null values
Column sq_ft has 0 null values
Column Unemployment_rate has 0 null values
Column income_rate has 0 null values
Column boro_Kings County                                       has 0 null values
Column boro_New York County                                    has 0 null values
Column boro_Queens County                                      has 0 null values
Column boro_Richmond County                                    has 0 null values


In [27]:
# List dataframe data types
new2_df.dtypes

price                                                        int64
zip                                                          int64
beds                                                        object
bath                                                       float64
sq_ft                                                        int64
Unemployment_rate                                          float64
income_rate                                                float64
boro_Kings County                                          float64
boro_New York County                                       float64
boro_Queens County                                         float64
boro_Richmond County                                       float64
dtype: object

In [28]:
new2_df['zip']= new2_df['zip'].astype('str')
new2_df.head()

,price,zip,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County
0,244000,11377,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0
1,680000,11367,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0
2,900000,11209,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0
3,625000,11420,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0
4,1720000,11361,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0


In [29]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
encz = OneHotEncoder(sparse=False,drop ="first")

In [30]:
# Fit the encoder and produce encoded DataFrame
zipone_df = pd.DataFrame(encz.fit_transform(new2_df.zip.values.reshape(-1,1)))

In [31]:
# Rename encoded columns
zipone_df.columns = encz.get_feature_names(['zip'])
zipone_df.head()

,zip_10002,zip_10003,zip_10004,zip_10005,zip_10006,zip_10007,zip_10009,zip_10010,zip_10011,zip_10012,...,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Merge the two DataFrames together and drop the boro column
new3_df=new2_df.merge(zipone_df,left_index=True,right_index=True).drop("zip",1)
new3_df.head()

,price,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County,...,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,244000,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,680000,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900000,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625000,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1720000,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
new3_df.dtypes

price                  int64
beds                  object
bath                 float64
sq_ft                  int64
Unemployment_rate    float64
                      ...   
zip_11432            float64
zip_11433            float64
zip_11434            float64
zip_11435            float64
zip_11436            float64
Length: 168, dtype: object

In [34]:
list(new3_df.iloc[2])

[900000,
 '3',
 1.0,
 1520,
 0.040644822060492754,
 0.15018151710082162,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.

In [35]:
new3_df['beds']= new3_df['beds'].astype('int')
new3_df.head()

,price,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County,...,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,244000,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,680000,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900000,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625000,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1720000,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
new3_df.dtypes

price                  int64
beds                   int64
bath                 float64
sq_ft                  int64
Unemployment_rate    float64
                      ...   
zip_11432            float64
zip_11433            float64
zip_11434            float64
zip_11435            float64
zip_11436            float64
Length: 168, dtype: object

In [37]:
# Print out the bath value counts
bath_counts = new3_df.bath.value_counts()
bath_counts

1.00     23310
2.00     18484
0.00     11973
3.00     10954
4.00      3267
5.00      1550
1.50       892
2.50       569
6.00       488
7.00       163
3.50       159
11.00       90
12.00       80
4.50        62
8.00        55
9.00        31
10.00       28
13.00       16
1.75        14
5.50        10
21.00        8
22.00        6
23.00        6
14.00        6
8.50         4
6.50         4
20.00        4
2.75         2
0.50         2
1.10         2
25.00        2
24.00        2
16.00        2
1.25         2
26.00        2
Name: bath, dtype: int64

In [38]:
# Print out the bath value counts
price_counts = new3_df.price.value_counts()

price_counts

10        916
750000    652
800000    636
650000    615
550000    592
         ... 
317554      1
11500       1
943400      1
834502      1
725237      1
Name: price, Length: 3744, dtype: int64

In [39]:
delete_row = new3_df[new3_df["price"]<11].index
new3_df = new3_df.drop(delete_row)
new3_df.head()

,price,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County,...,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,244000,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,680000,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900000,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625000,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1720000,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
scaled_features = new3_df.copy()

In [41]:
col_names = ['beds', 'bath','sq_ft', 'Unemployment_rate', 'income_rate']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

In [42]:
scaled_features[col_names] = features
scaled_features

,price,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County,...,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,244000,-1.135135,-0.518575,2.739442,-0.286425,-0.698632,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,680000,0.361618,0.854592,-0.153198,-0.286425,-0.698632,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900000,0.361618,-0.518575,-0.179076,2.044534,-0.595246,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625000,0.361618,0.168009,-0.183576,-0.286425,-0.698632,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1720000,-0.137300,-0.518575,-0.185489,-0.286425,-0.698632,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72439,640000,-1.135135,-1.205159,-0.123608,7.023896,0.434588,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72440,420000,1.858370,0.854592,-0.123496,7.023896,0.434588,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72441,485000,0.860535,0.168009,-0.123045,1.264579,0.434588,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72442,861000,0.860535,1.541176,-0.122820,1.264579,0.434588,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
col_name = ['price']
price = scaled_features['price']
scaler2 = StandardScaler().fit(price.values.reshape(-1,1))
price_feat = scaler2.transform(price.values.reshape(-1,1))
price_feat = pd.DataFrame(price_feat, columns=['price'])
price_feat

,price
0,-0.421745
1,-0.226940
2,-0.128644
3,-0.251514
4,0.237731
...,...
71148,-0.244812
71149,-0.343108
71150,-0.314066
71151,-0.146070


In [44]:
#scaled_features[col_names] = features
#scaled_features

In [45]:
list(scaled_features.columns)

['price',
 'beds',
 'bath',
 'sq_ft',
 'Unemployment_rate',
 'income_rate',
 'boro_Kings County                                      ',
 'boro_New York County                                   ',
 'boro_Queens County                                     ',
 'boro_Richmond County                                   ',
 'zip_10002',
 'zip_10003',
 'zip_10004',
 'zip_10005',
 'zip_10006',
 'zip_10007',
 'zip_10009',
 'zip_10010',
 'zip_10011',
 'zip_10012',
 'zip_10013',
 'zip_10014',
 'zip_10016',
 'zip_10017',
 'zip_10018',
 'zip_10019',
 'zip_10021',
 'zip_10022',
 'zip_10023',
 'zip_10024',
 'zip_10025',
 'zip_10026',
 'zip_10027',
 'zip_10028',
 'zip_10029',
 'zip_10030',
 'zip_10031',
 'zip_10032',
 'zip_10033',
 'zip_10034',
 'zip_10035',
 'zip_10036',
 'zip_10037',
 'zip_10038',
 'zip_10039',
 'zip_10040',
 'zip_10044',
 'zip_10065',
 'zip_10075',
 'zip_10128',
 'zip_10280',
 'zip_10301',
 'zip_10302',
 'zip_10303',
 'zip_10304',
 'zip_10305',
 'zip_10306',
 'zip_10307',
 'zip_10308'

In [46]:
# Split our preprocessed data into our features and target arrays
y = price_feat["price"].values
X = scaled_features.drop(["price"],1).values

In [47]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

Train and Evaluate Random Forest Classifier

In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 

In [49]:
# Create a random forest classifier.
rf_model = RandomForestRegressor(n_estimators=128,random_state=78)

In [50]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [51]:
# Evaluate the model
y_pred = rf_model.predict(X_test)
value = mean_squared_error(y_test,y_pred)
# calculate square
square = math.sqrt(value)
print(f" Housing price predictive root mean squared error: {square:.3f}")

 Housing price predictive root mean squared error: 0.785


In [52]:
# get importance
importance = rf_model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.04161
Feature: 1, Score: 0.08998
Feature: 2, Score: 0.56437
Feature: 3, Score: 0.00033
Feature: 4, Score: 0.04809
Feature: 5, Score: 0.00435
Feature: 6, Score: 0.04329
Feature: 7, Score: 0.00012
Feature: 8, Score: 0.00004
Feature: 9, Score: 0.00055
Feature: 10, Score: 0.00934
Feature: 11, Score: 0.00252
Feature: 12, Score: 0.00184
Feature: 13, Score: 0.00019
Feature: 14, Score: 0.00433
Feature: 15, Score: 0.00110
Feature: 16, Score: 0.00832
Feature: 17, Score: 0.00338
Feature: 18, Score: 0.00502
Feature: 19, Score: 0.00722
Feature: 20, Score: 0.00443
Feature: 21, Score: 0.00179
Feature: 22, Score: 0.00198
Feature: 23, Score: 0.00214
Feature: 24, Score: 0.01462
Feature: 25, Score: 0.00522
Feature: 26, Score: 0.00339
Feature: 27, Score: 0.00121
Feature: 28, Score: 0.00302
Feature: 29, Score: 0.00186
Feature: 30, Score: 0.00131
Feature: 31, Score: 0.00221
Feature: 32, Score: 0.00222
Feature: 33, Score: 0.00165
Feature: 34, Score: 0.00049
Feature: 35, Score: 0.00211
Fe

In [53]:
predictions2 = rf_model.predict(X_test)
pd.DataFrame({"Prediction": predictions2, "Actual": y_test})

,Prediction,Actual
0,0.450205,0.318155
1,-0.275918,-0.300215
2,-0.280524,-0.269386
3,-0.207478,-0.240344
4,1.774755,2.060672
...,...,...
17784,0.028330,-0.113006
17785,0.113798,-0.095135
17786,-0.254720,-0.086199
17787,0.519878,0.246667


In [54]:
baseline_preds = np.array([y_test.mean() for _ in range(len(y_test))]) 
baseline_preds

array([-0.00363385, -0.00363385, -0.00363385, ..., -0.00363385,
       -0.00363385, -0.00363385])

In [55]:
value2 = mean_squared_error(baseline_preds,y_test)
value2

0.9630485090114534

In [56]:
value3 = mean_squared_error(y_test,predictions2)
value3


0.6158539896722055

Linear Regression

In [57]:
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier

LinearRegression()

In [58]:
classifier.fit(X_train, y_train)

LinearRegression()

In [59]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
0,0.773193,0.318155
1,-0.408691,-0.300215
2,-0.200439,-0.269386
3,-0.164551,-0.240344
4,0.953369,2.060672
...,...,...
17784,0.258057,-0.113006
17785,0.613037,-0.095135
17786,-0.163330,-0.086199
17787,0.640381,0.246667


In [60]:
print(classifier.coef_)

[-1.47870067e-02  1.59260035e-01  4.92108370e-02 -3.97301493e+11
  2.20187848e+12  2.89261724e+11 -5.17578587e+12 -4.09188312e+11
 -2.28818503e+12 -4.32296753e-01 -1.28845215e-01 -1.10187531e-02
 -4.26879883e-01 -4.73624706e-01  1.35620117e-01 -3.55590820e-01
 -1.24465942e-01 -2.17773438e-01  2.10418701e-01  2.08679199e-01
 -1.97753906e-02 -4.37316895e-01 -4.35302734e-01  5.07888794e-02
 -1.58203125e-01 -2.55523682e-01 -3.15162659e-01 -3.85284424e-01
 -1.31698608e-01 -4.48120117e-01 -5.05981445e-01 -4.45922852e-01
 -2.79205322e-01 -4.54772949e-01 -5.53710938e-01 -4.95086670e-01
 -5.06591797e-01 -5.01480103e-01 -6.17687225e-01 -3.31604004e-01
 -4.10293579e-01 -5.77987671e-01 -4.48852539e-01 -6.35284424e-01
 -5.80112457e-01 -6.00479126e-01 -6.51786804e-01 -4.09664154e-01
 -7.10449219e-01 -4.97014046e-01 -2.21145630e-01 -2.19383240e-01
 -2.15484619e-01 -2.41172791e-01 -2.52944946e-01 -2.34237671e-01
 -1.90666199e-01 -2.32376099e-01 -2.33932495e-01 -2.64015198e-01
 -2.18750000e-01 -2.09823

In [61]:
classifier.intercept_

1833693758801.741

In [62]:
X_test[0]

array([-0.63621744,  0.16800858, -0.18531993, -0.09751796,  1.50024087,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [63]:
pred_test = classifier.predict(np.expand_dims(X_test[0],0))

In [83]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

Mean Absolute Error: 0.33502608334646894
Mean Squared Error: 0.8323554421898676
Root Mean Squared Error: 0.9123351589135802


In [65]:
#import pickle

In [66]:
#pickle.dump(rf_model, open('model.pkl','wb'))
#rf_model = pickle.load(open('model.pkl','rb'))

nn


In [68]:
#Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  300
hidden_nodes_layer2 = 24


In [69]:
nn_model = tf.keras.models.Sequential()

In [70]:
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation = "relu", input_dim = number_input_features))

In [71]:
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

In [72]:
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

In [73]:
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               50400     
_________________________________________________________________
dense_1 (Dense)              (None, 24)                7224      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25        
Total params: 57,649
Trainable params: 57,649
Non-trainable params: 0
_________________________________________________________________


In [79]:
nn_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
nn_model.fit(X_train, np.asarray(y_train), epochs=100)

Epoch 1/100
1668/1668 [==============================] - 1s 710us/step - loss: 0.5689 - accuracy: 0.0000e+000s - loss: 0.5871 - ac
Epoch 2/100
1668/1668 [==============================] - 1s 705us/step - loss: 0.5663 - accuracy: 0.0000e+00
Epoch 3/100
1668/1668 [==============================] - 1s 691us/step - loss: 0.5533 - accuracy: 0.0000e+00
Epoch 4/100
1668/1668 [==============================] - 1s 697us/step - loss: 0.5546 - accuracy: 0.0000e+00
Epoch 5/100
1668/1668 [==============================] - 1s 720us/step - loss: 0.5409 - accuracy: 0.0000e+00
Epoch 6/100
1668/1668 [==============================] - 1s 710us/step - loss: 0.5381 - accuracy: 0.0000e+00
Epoch 7/100
1668/1668 [==============================] - 1s 719us/step - loss: 0.5260 - accuracy: 0.0000e+00
Epoch 8/100
1668/1668 [==============================] - 1s 695us/step - loss: 0.5255 - accuracy: 0.0000e+00
Epoch 9/100
1668/1668 [==============================] - 1s 694us/step - loss: 0.5144 - accuracy: 0.0000e+

1668/1668 [==============================] - 1s 758us/step - loss: 0.3579 - accuracy: 0.0000e+00
Epoch 76/100
1668/1668 [==============================] - 1s 765us/step - loss: 0.3246 - accuracy: 0.0000e+00
Epoch 77/100
1668/1668 [==============================] - 1s 794us/step - loss: 0.3356 - accuracy: 0.0000e+00
Epoch 78/100
1668/1668 [==============================] - 1s 848us/step - loss: 0.3399 - accuracy: 0.0000e+00
Epoch 79/100
1668/1668 [==============================] - 1s 766us/step - loss: 0.3256 - accuracy: 0.0000e+00
Epoch 80/100
1668/1668 [==============================] - 1s 734us/step - loss: 0.3260 - accuracy: 0.0000e+00
Epoch 81/100
1668/1668 [==============================] - 1s 739us/step - loss: 0.3368 - accuracy: 0.0000e+00
Epoch 82/100
1668/1668 [==============================] - 1s 713us/step - loss: 0.3334 - accuracy: 0.0000e+00
Epoch 83/100
1668/1668 [==============================] - 1s 716us/step - loss: 0.3281 - accuracy: 0.0000e+00
Epoch 84/100
1668/1668 

In [80]:
y_train_pred = nn_model.predict(X_train)
y_test_pred = nn_model.predict(X_test)

In [81]:
r2_score(y_train, y_train_pred)

0.6907235855379542

In [77]:
r2_score(y_test, y_test_pred)

0.2931637397859298

In [ ]:
import pickle

# all supressed so we dont accidently override the current file.

#pickle.dump(rf_model, open('rfmodel.pkl','wb'))
#rf_model = pickle.load(open('rfmodel.pkl','rb'))
#pickle.dump(scaler, open('scaler.pkl','wb'))
#pickle.dump(scaler2, open('scaler2.pkl','wb'))
#pickle.dump(enc, open('encoder.pkl','wb'))
#pickle.dump(encz, open('encoderz.pkl','wb'))

In [ ]:
#un-scale the y value
y_inverse = scaler2.inverse_transform(pred_test)
y_inverse